In [19]:
import pandas as pd
import time
import datetime
import numpy as np
from datetime import date
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer, log_loss

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
import imblearn
from imblearn.over_sampling import SMOTE

In [2]:
classify = pd.read_csv('https://raw.githubusercontent.com/shubhadipbiswas/Classification_heroku/master/Booking_Classification.csv')

In [3]:
classify.head()

,Hotel_Type,lead_time,Date_Year,Date_Month,Year_Week,Day_Of_Month,Weekend_Stay?,stays_in_week_nights,Count_Pax,Count_Minors,Infants,Boarding,country,Booking_Mode,Booking_Channel,Repeat_Customer?,previous_cancellations?,previous_bookings_not_canceled?,Cat_Type1,Cat_Type2,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,Booking_Cancelled
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015,0
1,Resort Hotel,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015,0
2,Resort Hotel,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015,0
3,Resort Hotel,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015,0
4,Resort Hotel,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,03-07-2015,0


In [4]:
df = classify.copy()

In [5]:
df['month'] = pd.to_datetime(df.Date_Month, format='%B').dt.month
cols=["Date_Year","month","Day_Of_Month"]
df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
df['date']=pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.dayofweek
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['Weekend'] = df.weekday.isin([5, 6]).astype(int)
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
#df['weekofyear'] = df['date'].dt.isocalendar().week

In [6]:
target = ['Booking_Cancelled']
zeroencode = ['Hotel_Type','Boarding','Booking_Mode','Booking_Channel','Cat_Type1','Cat_Type2','deposit_type','customer_type']
drops=['country', 'Day_Of_Month','Date_Month','date', 'agent', 'company', 'reservation_status', 'reservation_status_date']

In [7]:
df=df.drop(drops, axis = 1) 

In [8]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Hotel_Type                       119390 non-null  object 
 1   lead_time                        119390 non-null  int64  
 2   Date_Year                        119390 non-null  int64  
 3   Year_Week                        119390 non-null  int64  
 4   Weekend_Stay?                    119390 non-null  int64  
 5   stays_in_week_nights             119390 non-null  int64  
 6   Count_Pax                        119390 non-null  int64  
 7   Count_Minors                     119386 non-null  float64
 8   Infants                          119390 non-null  int64  
 9   Boarding                         119390 non-null  object 
 10  Booking_Mode                     119390 non-null  object 
 11  Booking_Channel                  119390 non-null  object 
 12  Re

<ipython-input-8-acd3301a9435>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(null_counts=True)


### Encoding Categorical Variables

In [9]:
df= pd.get_dummies(df, columns=zeroencode)


### MinMaxScaler() to Normalize Data

In [10]:
min_max_scaler = preprocessing.MinMaxScaler()
df['lead_time']= min_max_scaler.fit_transform(df[['lead_time']])
df['adr']= min_max_scaler.fit_transform(df[['adr']])

In [11]:
df['Booking_Cancelled'].value_counts()

0    75166
1    44224
Name: Booking_Cancelled, dtype: int64

In [12]:
df.head()

,lead_time,Date_Year,Year_Week,Weekend_Stay?,stays_in_week_nights,Count_Pax,Count_Minors,Infants,Repeat_Customer?,previous_cancellations?,previous_bookings_not_canceled?,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,Booking_Cancelled,month,weekday,Weekend,quarter,Hotel_Type_City Hotel,Hotel_Type_Resort Hotel,Boarding_BB,Boarding_FB,Boarding_HB,Boarding_SC,Boarding_Undefined,Booking_Mode_Aviation,Booking_Mode_Complementary,Booking_Mode_Corporate,Booking_Mode_Direct,Booking_Mode_Groups,Booking_Mode_Offline TA/TO,Booking_Mode_Online TA,Booking_Mode_Undefined,Booking_Channel_Corporate,Booking_Channel_Direct,Booking_Channel_GDS,Booking_Channel_TA/TO,Booking_Channel_Undefined,Cat_Type1_A,Cat_Type1_B,Cat_Type1_C,Cat_Type1_D,Cat_Type1_E,Cat_Type1_F,Cat_Type1_G,Cat_Type1_H,Cat_Type1_L,Cat_Type1_P,Cat_Type2_A,Cat_Type2_B,Cat_Type2_C,Cat_Type2_D,Cat_Type2_E,Cat_Type2_F,Cat_Type2_G,Cat_Type2_H,Cat_Type2_I,Cat_Type2_K,Cat_Type2_L,Cat_Type2_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0.464043,2015,27,0,0,2,0.0,0,0,0,0,3,0,0.001180,0,0,0,7,2,0,3,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1.000000,2015,27,0,0,2,0.0,0,0,0,0,4,0,0.001180,0,0,0,7,2,0,3,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0.009498,2015,27,0,1,1,0.0,0,0,0,0,0,0,0.015053,0,0,0,7,2,0,3,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,0.017639,2015,27,0,1,1,0.0,0,0,0,0,0,0,0.015053,0,0,0,7,2,0,3,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,0.018996,2015,27,0,2,2,0.0,0,0,0,0,0,0,0.019307,0,1,0,7,2,0,3,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [13]:
df=df.replace([np.inf, -np.inf], np.nan)
df=df.fillna(0)

In [14]:
X = df.drop(target, axis=1)
Y = df[target]

In [15]:
X_col=list(X.columns)
Y_col=list(Y.columns)

#### Using SMOTE to balance the class

In [16]:
os = SMOTE(random_state=0)
os_data_X,os_data_y=os.fit_resample(X, Y)
X = pd.DataFrame(data=os_data_X,columns=X_col )
Y= pd.DataFrame(data=os_data_y,columns=Y_col)

#### Let's Generate 5 model – Logistic regression, SVM, KNN , Naïve Bayes, Random forest.

In [25]:
logreg = LogisticRegression()
sv = LinearSVC()
knn = KNeighborsClassifier(n_neighbors = 8)
NB = GaussianNB()
rf = RandomForestClassifier(random_state = 1)

In [27]:
#accuracy=('Accuracy : {:.2f}'
#     .format(sv.score(X, Y)))

In [28]:
#accuracy

In [29]:
# Train and evaluate model
def fit_eval_model(model, X,Y):
    
 
    results = {}
    
    # Train the model
    model.fit(X, Y)
    
    # Test the model
    predicted = model.predict(X)
    
    
     # Evaluation
    results['classification_report'] = classification_report(Y, predicted)
    results['confusion_matrix'] = confusion_matrix(Y, predicted)
    results['accuracy']=('Accuracy : {:.2f}'
     .format(model.score(X, Y)))
    results['predicted'] = model.predict(X)
    results['Logloss']= log_loss(Y.values, predicted)
        
    return results

In [30]:
# Fit and evaluate models
results = {}
for cls in [logreg, sv, knn, NB,rf]:
    cls_name = cls.__class__.__name__
    results[cls_name] = {}
    results[cls_name] = fit_eval_model(cls, X,Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel

In [31]:
# Print Results of Models
for result in results:
    print (result)
    print()
    for i in results[result]:
        print (i, ':')
        print(results[result][i])
        print()
    print ('-----')
    print()

LogisticRegression

classification_report :
              precision    recall  f1-score   support

           0       0.73      0.77      0.75     75166
           1       0.76      0.72      0.74     75166

    accuracy                           0.74    150332
   macro avg       0.75      0.74      0.74    150332
weighted avg       0.75      0.74      0.74    150332


confusion_matrix :
[[57737 17429]
 [20946 54220]]

accuracy :
Accuracy : 0.74

predicted :
[0 0 0 ... 1 0 1]

Logloss :
8.816748798248598

-----

LinearSVC

classification_report :
              precision    recall  f1-score   support

           0       0.65      0.98      0.78     75166
           1       0.95      0.47      0.63     75166

    accuracy                           0.72    150332
   macro avg       0.80      0.72      0.70    150332
weighted avg       0.80      0.72      0.70    150332


confusion_matrix :
[[73409  1757]
 [39928 35238]]

accuracy :
Accuracy : 0.72

predicted :
[0 0 0 ... 0 0 1]

Logloss :

In [ ]:
# RandomForestClassifier provided best accuracy of 0.99

###### Final Model RandomForestClassifier

In [33]:
rf = RandomForestClassifier(random_state = 1)
rf.fit(X, Y)

<ipython-input-33-d31c261dbe30>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X, Y)


RandomForestClassifier(random_state=1)

In [34]:
# Saving the model
import pickle
pickle.dump(rf, open('clas_rf.pkl', 'wb'))